<a href="https://colab.research.google.com/github/goldiezhu/BIS634/blob/main/A4/E4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:

# Smith - Watemran Algorithm
import numpy as np

### PART 1 ###
# implement function that takes two strings and uses the Smith-Waterman Algorithm
# to return an optimal local alignment and score
# insert '-' to indicate gap 
# take three keyword arguments with default 1 
# (penalty of one applied to match scores for each missing or changed letter)

    
def smith_waterman(seq1, seq2, match = 1, gap_penalty = 1, mismatch_penalty = 1):
    length_m = len(seq1)
    length_n = len(seq2)
    compute_val = 0
    max_score = 0
    max_score_m = 0
    max_score_n = 0
    
   # init matrix to zero, have an extra row at top and extra col on left
    matrix = np.zeros((length_m+1, length_n+1), np.int)
    ### First half of algo: Make the Matrix 
    for m in range(1, length_m+1):
        for n in range(1, length_n+1):
            # if match found
            if (seq1[m-1] == seq2[n-1]):
                # upper left + match
                compute_val = matrix[m-1][n-1] + match
            # if match not found
            else:
                # upper left - mismatch penalty
                compute_val = matrix[m-1][n-1] - mismatch_penalty

            # find actual value to put into matrix
            matrix[m][n] = max(compute_val, matrix[m][n-1] - gap_penalty, matrix[m-1][n] - gap_penalty, 0)
            
            # check max score
            if (matrix[m][n] > max_score):
                max_score = matrix[m][n]
                # add 1 to max scores to account for the 0 index
                max_score_m = m
                max_score_n = n
   
    ### Second half of algo: Backtracking from max value 
    # prioritizing gap (insertion and deletions), and not mismatch
    
    # corresponding seq element from max value
    tb_m = max_score_m
    tb_n = max_score_n
    match_seq1 = ""
    match_seq2 = ""
    while (matrix[tb_m][tb_n] > 0):
        if ((seq1[tb_m-1] == seq2[tb_n-1]) and matrix[tb_m-1][tb_n-1] == matrix[tb_m][tb_n] - match):
            match_seq1 = seq1[tb_m-1] + match_seq1
            match_seq2 = seq2[tb_n-1] + match_seq2
            # shift up and to the left
            tb_m -= 1 
            tb_n -= 1
        # If not a match (prioritize gaps, not mismatches)
        else:
            # current = l - gap
            if (matrix[tb_m][tb_n] == matrix[tb_m][tb_n-1] - gap_penalty):
                match_seq1 = '-' + match_seq1
                match_seq2 = seq2[tb_n-1] + match_seq2
                # shift left
                tb_n-= 1
            # current = up - gap
            elif (matrix[tb_m][tb_n] == matrix[tb_m-1][tb_n] - gap_penalty):
                match_seq1 = seq1[tb_m-1] + match_seq1
                match_seq2 = '-' + match_seq2
                # shift up
                tb_m -= 1
            else:
                tb_m -= 1 
                tb_n -= 1
    return match_seq1, match_seq2, max_score  


### PART 2 ###
# Test it, and explain how tests show the function works. Test other values.

# Examples from the problem statement:
sequence1, sequence2, score = smith_waterman('tgcatcgagaccctacgtgac', 'actagacctagcatcgac')
print('Sequence 1: {}, Sequence 2: {}, Score: {}'.format(sequence1, sequence2, score))
sequence1, sequence2, score = smith_waterman('tgcatcgagaccctacgtgac', 'actagacctagcatcgac', gap_penalty=2)
print('Sequence 1: {}, Sequence 2: {}, Score: {}'.format(sequence1, sequence2, score))

# Example from the cheatsheet
sequence1, sequence2, score = smith_waterman('gttacc', 'gttgac')
print('Sequence 1: {}, Sequence 2: {}, Score: {}'.format(sequence1, sequence2, score))

# To test whether or not the above smith-waterman function is correct, I will manipulate the parameters.

# Here is the control:
sequence1, sequence2, score = smith_waterman('gacttac', 'cgtgaattcat', match = 5, gap_penalty = 4, mismatch_penalty = 3)
print('Sequence 1: {}, Sequence 2: {}, Score: {}'.format(sequence1, sequence2, score))

# Now I wil increase the match value. If I increase the match value, I expect score to increase because I'm rewarding more for matching nucleotides.
sequence1, sequence2, score = smith_waterman('gacttac', 'cgtgaattcat', match = 6, gap_penalty = 4, mismatch_penalty = 3)
print('Sequence 1: {}, Sequence 2: {}, Score: {}'.format(sequence1, sequence2, score))
# The score increased from 18 to 23.

# Starting from the control again, I will now increase only the gap_penalty value. This should decrease the score.
sequence1, sequence2, score = smith_waterman('gacttac', 'cgtgaattcat', match = 5, gap_penalty = 5, mismatch_penalty = 3)
print('Sequence 1: {}, Sequence 2: {}, Score: {}'.format(sequence1, sequence2, score))
# The score decreased from 18 to 17.

# Starting from the control again, I will now increase only the mismatch_penalty value. This should also decrease the score.
sequence1, sequence2, score = smith_waterman('gacttac', 'cgtgaattcat', match = 5, gap_penalty = 4, mismatch_penalty = 4)
print('Sequence 1: {}, Sequence 2: {}, Score: {}'.format(sequence1, sequence2, score))
# The score decreased from 18 to 17.


Sequence 1: agacccta-ct-gac, Sequence 2: aga-cctagctcgac, Score: 8
Sequence 1: gcatcga, Sequence 2: gcatcga, Score: 7
Sequence 1: gtt-ac, Sequence 2: gttgac, Score: 4
Sequence 1: gatt-a, Sequence 2: gattca, Score: 18
Sequence 1: gatt-a, Sequence 2: gattca, Score: 23
Sequence 1: gatt, Sequence 2: gatt, Score: 17
Sequence 1: gatt-a, Sequence 2: gattca, Score: 17


<ipython-input-11-d43ffd97d558>:21: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  matrix = np.zeros((length_m+1, length_n+1), np.int)
